In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# ヘッドレスオプション
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://interview.interpresident.jp/")
time.sleep(2)

article_urls = set()

while True:
    # 現在のページからa.index-article-aを取得
    soup = BeautifulSoup(driver.page_source, "html.parser")
    a_tags = soup.select("a.index-article-a")
    print(f"{len(a_tags)} 件のリンクを取得")
    for a in a_tags:
        href = a.get("href")
        if href:
            full_url = "https://interview.interpresident.jp" + href if href.startswith("/") else href
            article_urls.add(full_url)

    # 「次へ」リンクを検出
    try:
        # 現在アクティブなページ番号を探す
        current = driver.find_element("css selector", ".page-numbers.current")
        current_num = int(current.text)

        # すべてのページ番号を取得
        page_links = driver.find_elements("css selector", ".page-numbers")

        # 次のページがあるかチェック
        next_page = None
        for link in page_links:
            try:
                if int(link.text) == current_num + 1:
                    next_page = link
                    break
            except:
                continue  # テキストが数字でない（例：「次へ」など）の場合はスキップ

        if next_page:
            next_page.click()
            time.sleep(2)
        else:
            print("次のページが見つかりません。終了します。")
            break
    except Exception as e:
        print("ページ遷移中にエラーが発生：", e)
        break

driver.quit()

# 結果を出力
print("\n✅ 全URL一覧（{}件）".format(len(article_urls)))
for url in sorted(article_urls):
    print(url)
import csv

# CSVに保存
with open("interview_links.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["URL"])  # ヘッダー
    for url in sorted(article_urls):
        writer.writerow([url])

print("\n✅ CSVファイル 'interview_links.csv' に保存完了！")

21 件のリンクを取得
21 件のリンクを取得
21 件のリンクを取得
6 件のリンクを取得
次のページが見つかりません。終了します。

✅ 全URL一覧（69件）
https://interview.interpresident.jp/kansai/chatwork/
https://interview.interpresident.jp/kansai/daikoku/
https://interview.interpresident.jp/kansai/daimei/
https://interview.interpresident.jp/kansai/eco-style/
https://interview.interpresident.jp/kansai/ewell/
https://interview.interpresident.jp/kansai/family-network/
https://interview.interpresident.jp/kansai/hanamaru/
https://interview.interpresident.jp/kansai/i-plug/
https://interview.interpresident.jp/kansai/kdl/
https://interview.interpresident.jp/kansai/lockon/
https://interview.interpresident.jp/kansai/meryeself/
https://interview.interpresident.jp/kansai/minatoseiki/
https://interview.interpresident.jp/kansai/mwcc/
https://interview.interpresident.jp/kansai/nikko/
https://interview.interpresident.jp/kansai/nissonet/
https://interview.interpresident.jp/kansai/noah/
https://interview.interpresident.jp/kansai/raxus-create/
https://interview.interpres

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import csv

# ファイル設定
input_file = "interview_links.csv"
output_file = "company_info_result.csv"

headers = {'User-Agent': 'Mozilla/5.0'}
df_links = pd.read_csv(input_file)
results = []

for url in df_links["URL"]:
    try:
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # 初期値
        company = "N/A"
        person = "N/A"
        company_url = "N/A"

        # 1. 通常の company-info クラスから取得
        name_tag = soup.select_one(".company-info__name")
        if name_tag:
            company = name_tag.text.strip()

        person_tag = soup.select_one(".company-info__person")
        if person_tag:
            person = person_tag.text.strip()

        url_tag = soup.select_one(".company-info__url a")
        if url_tag:
            company_url = url_tag.get("href").strip()

        # 2. 上記で取れなかった場合、company-info-table から取得
        if company == "N/A" or person == "N/A" or company_url == "N/A":
            rows = soup.select(".company-info-table tr")
            for row in rows:
                th = row.select_one("th")
                td = row.select_one("td")
                if not th or not td:
                    continue
                label = th.text.strip()
                value = td.text.strip()

                if company == "N/A" and "会社名" in label:
                    company = value
                elif person == "N/A" and ("代表者" in label or "氏名" in label):
                    person = value
                elif company_url == "N/A" and ("ＵＲＬ" in label or "HP" in label):
                    company_url = value

        results.append([url, company, person, company_url])
        print(f"✅ 取得: {url}")
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ エラー: {url} -> {e}")
        results.append([url, "Error", "Error", str(e)])

# CSVに保存
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["記事URL", "会社名", "代表者名", "会社URL"])
    writer.writerows(results)

print(f"\n✅ 完了: {output_file} に保存されました。")


✅ 取得: https://interview.interpresident.jp/kansai/chatwork/
✅ 取得: https://interview.interpresident.jp/kansai/daikoku/
✅ 取得: https://interview.interpresident.jp/kansai/daimei/
✅ 取得: https://interview.interpresident.jp/kansai/eco-style/
✅ 取得: https://interview.interpresident.jp/kansai/ewell/
✅ 取得: https://interview.interpresident.jp/kansai/family-network/
✅ 取得: https://interview.interpresident.jp/kansai/hanamaru/
✅ 取得: https://interview.interpresident.jp/kansai/i-plug/
✅ 取得: https://interview.interpresident.jp/kansai/kdl/
✅ 取得: https://interview.interpresident.jp/kansai/lockon/
✅ 取得: https://interview.interpresident.jp/kansai/meryeself/
✅ 取得: https://interview.interpresident.jp/kansai/minatoseiki/
✅ 取得: https://interview.interpresident.jp/kansai/mwcc/
✅ 取得: https://interview.interpresident.jp/kansai/nikko/
✅ 取得: https://interview.interpresident.jp/kansai/nissonet/
✅ 取得: https://interview.interpresident.jp/kansai/noah/
✅ 取得: https://interview.interpresident.jp/kansai/raxus-create/
✅ 取得: ht